In [ ]:
#This code has been inspired by the works of:
# 1: ugik (2017) Tensorflow chat-bot model.ipynb Available At: https://github.com/ugik/notebooks/blob/master/Tensorflow%20chat-bot%20model.ipynb (Accessed: 2/02/2020)
# 2: FreeBirdsCrew (2020) CHATBOTS - Using Natural Language Processing and Tensorflow Available At: https://github.com/FreeBirdsCrew/AI_ChatBot_Python/blob/master/Contextual%20Chatbot%20-%20NLP%20and%20Tensorflow.ipynb (Accessed: 6/02/2020)
# 3: Tech With Tim (2019) Python Chat Bot Tutorial - Chatbot with Deep Learning (Part 3) Available At: https://www.youtube.com/watch?v=PzzHOvpqDYs&ab_channel=TechWithTim (Accessed: 16/10/2020)
# 4: Tech With Tim (2019) Python Chat Bot Tutorial - Chatbot with Deep Learning (Part 2) Available At: https://www.youtube.com/watch?v=ON5pGUJDNow (Accessed: 16/10/2020)

In [ ]:
!python --version

Python 3.7.10


In [ ]:
pip install tflearn

In [ ]:
pip install tensorflow==1.14.0

In [ ]:
!pip install flask-ngrok

In [ ]:
from IPython.display import Javascript

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import tensorflow as tf
import tflearn
import numpy as np
import random
import json
import nltk
import pickle
import math
import binascii
import webbrowser
import urllib.error
import urllib.request
import smtplib
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import time
import os
import sys
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [ ]:
tf.version.VERSION

'1.14.0'

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
qaCode = binascii.unhexlify("5468616e6b20796f7520666f722070617274696369706174696e6720666f72203130206d696e75746573212054686520436f646520666f7220746865207175657374696f6e6e61697265206973204135337274593133322120596f752063616e206b6565702074657374696e6720696620796f752077616e743a203c62722f3e3c62722f3e")

In [ ]:
try:
  with open('intents.json') as json_data:
      intents = json.load(json_data)
except FileNotFoundError:
  print("json file not found")
  sys.exit('Closing Program')

In [ ]:
files = ["./templates/index.html", "./static/pop.mp3", "./static/css/style.css"]
for item in files:
  if os.path.isfile(item):
    continue
  else:
    print(item + " does not exist in the correct directory! Please refer back to the guidelines found in the email")
    sys.exit('Closing Program')

In [ ]:
words = []
classes = []
documents = []

In [ ]:
#This section is designed to loop through each pattern in an intent
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #This section is designed for tokenization: This will be proccessed to all words in a sentence
        try:
          w = nltk.word_tokenize(pattern)
        except LookupError as ntlkerror:
          print("Error during tokenization: ", ntlkerror)
          sys.exit('Closing Program')

        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
#This section is designed for stemming each word
#This section will also ensure that any capital letter will be tranformed to lower case
try:
  words = [stemmer.stem(w.lower()) for w in words if w not in "?"]
  words = sorted(list(set(words)))
except IndexError:
  print("Error stemming words")
  sys.exit('Closing Program')

In [ ]:

classes = sorted(list(set(classes)))

In [ ]:
training = []
output = []
output_empty = [0] * len(classes)

In [ ]:
# This section is designed to transform each sentence into a BOW[Bag of Words]
for doc in documents:
    bag_of_words = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

    for w in words:
      if w in pattern_words:
        bag_of_words.append(1)
      else:
        bag_of_words.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag_of_words, output_row])

In [ ]:
random.shuffle(training)
training = np.array(training)

In [ ]:
# This will create our training and testing data as lists
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [ ]:
try:
  tf.reset_default_graph()
except AttributeError as tfgrapherror:
  print("Error resetting tensorflow graph: ", tfgrapherror)
  sys.exit('Closing Program')

In [ ]:
try:
  hiddenLayerNode = math.ceil(((len(train_x[0]) + len(train_y[0])) / 2))
  net = tflearn.input_data(shape=[None, len(train_x[0])])
  net = tflearn.fully_connected(net, hiddenLayerNode)
  net = tflearn.fully_connected(net, math.ceil((hiddenLayerNode / 2)))
  net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
  net = tflearn.regression(net)
except ValueError as nnbuilderror:
  print("Error creating the neural network model: ", nnbuilderror)
  sys.exit('Closing Program')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
#If path is not in directory try to create folder ./tflearnModel
if not os.path.isdir('./tflearnModel'):
    try:
        os.mkdir("./tflearnModel")
    except OSError:
        print("Creation of the directory failed")
        sys.exit('Closing Program')
    else:
        print("Successfully created the directory")
        

Successfully created the directory


In [ ]:
# This section is designed to created our neural networking model
try:
  model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
  # Start training (apply gradient descent algorithm)
  model.fit(train_x, train_y, n_epoch=100, batch_size=32, show_metric=True)
  model.save('./tflearnModel/model.tflearn')
except IndexError as nnfiterror:
  print("Error fitting the neural network model: ", nnfiterror)
  sys.exit('Closing Program')

Training Step: 2499  | total loss: 0.00276 | time: 0.195s
| Adam | epoch: 100 | loss: 0.00276 - acc: 0.9993 -- iter: 768/782
Training Step: 2500  | total loss: 0.00255 | time: 0.201s
| Adam | epoch: 100 | loss: 0.00255 - acc: 0.9994 -- iter: 782/782
--
INFO:tensorflow:/content/tflearnModel/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
#Try to pickle words/classes and training data as to convert and save  words/classes and training data into bytes.
try:
  pickle.dump({'words': words, 'classes': classes, 'train_x': train_x, 'train_y': train_y},
              open("./tflearnModel/model.tflearn", "wb"))
except (IOError, OSError, pickle.PickleError, pickle.UnpicklingError):
  print("Error writing/saving pickled data as tflearn model")
  sys.exit("Closing Program")
except FileNotFoundError:
  print("tflearn model does not exist")
  sys.exit("Closing Program")

In [ ]:
try:
  data = pickle.load(open("./tflearnModel/model.tflearn", "rb"))
except (IOError, OSError, pickle.PickleError, pickle.UnpicklingError):
  print("Error loading pickled data from tflearn model")
  sys.exit("Closing Program")
except FileNotFoundError:
  print("tflearn model does not exist")
  sys.exit("Closing Program")

words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [ ]:
class ContextHistory:
    contextOld = ""
    myIntent = ""
    myOldTag = ""
    myTag = ""
    myFirstContext = ""

    myFirstInput = 0

    myEmailIntent = 0
    studentEmail = ""
    studentPassword = ""
    staffEmail = ""
    symptomType = ""
    symptomProblem = ""

    time = 0.0
    startTime = 0.0

    chatRestart = "<br/><br/>Please specify if you want " \
                  "guidance for your <fears> or anxiety <symptoms> whilst at Cardiff Met during the coronavirus " \
                  "epidemic. Input (fears) or (symptoms)"
myOldContext = ContextHistory()

In [ ]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
model.load('./tflearnModel/model.tflearn')

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/tflearnModel/model.tflearn


In [ ]:
def clean_up_sentence(sentence):
    #This is designed to tokenize the  user input
    sentence_words = nltk.word_tokenize(sentence)
    #This is designed to stem the words from the users inputs
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [ ]:
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag_of_words = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag_of_words[i] = 1
                if show_details:
                    print("found in bag: %s" % w)

    return np.array(bag_of_words)

In [ ]:
def myTime():
    if ContextHistory.myFirstInput < 1:
        ContextHistory.startTime = time.perf_counter()

    ContextHistory.time = (time.perf_counter() - ContextHistory.startTime)
    print(ContextHistory.time)

In [ ]:
context = {}

In [ ]:
ERROR_THRESHOLD = 0.70

In [ ]:
def classify(sentence):
    #This section is designed to calculate the probability values from the users input to the model
    results = model.predict([bow(sentence, words)])[0]
    #If a prediction is below the error thresehold of 0.70 then it is filtered out
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    #Calculates and sorts the probability rates comparing to its strength/probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))

    return return_list

In [ ]:
def filtering(i, show_details):
    if 'context_set' in i:
        if show_details:
            return 'context:', i['context_set']
        myOldContext.contextOld = i['context_set']
        myOldContext.myTag = i['tag']
        return i['context_set']

In [ ]:
def advancedResponse(i):
    if 'web' in i:
        myWeb = i['web']
        try:
            urllib.request.urlopen(myWeb)
        except urllib.error.HTTPError:
            return "I am sorry but I am having trouble connecting to the site: " + random.choice(i['responses'])
        except urllib.error.URLError:
            return "I am sorry but I am having trouble connecting to the site" + random.choice(i['responses'])
        else:
            url = myWeb
            display(Javascript('window.open("{url}");'.format(url=url)))
            return "Redirecting: " + random.choice(i['responses'])

    else:
        return random.choice(i['responses'])

In [ ]:
def email(userInput):
    if myOldContext.myEmailIntent == 1:
        myOldContext.studentEmail = str(userInput) + "@outlook.cardiffmet.ac.uk"
        print(str(userInput) + "from email")
        myOldContext.myEmailIntent += 1
        return "Please Enter Your Student Password"
    else:
        recEmail = myOldContext.staffEmail
        subject = myOldContext.studentEmail + " " + myOldContext.symptomType + " anxiety symptom issues"
        body = "Dear Sir/Madam,\n\n This student has been recorded at a high priority as the participant has shown " \
               "signs of suffering from severe " + myOldContext.symptomType + " anxiety symptoms. The symptoms " \
                                                                              "specified indicate that the " \
                                                                              "student could be suffering from " \
                                                                              "anxities that can lead to " + \
               myOldContext.symptomProblem + "\n\nPlease Email them as soon as possible as to arrange a formal meetup " \
                                             "and/or conversation. \n\n Kindest Regards\nMetMind Chatbot"
        msg = f'Subject: {subject}\n\n{body}'

        myOldContext.studentPassword = str(userInput)

        try:
            server = smtplib.SMTP('smtp.outlook.com', 587)
            server.starttls()
            server.login(myOldContext.studentEmail, myOldContext.studentPassword)
            server.sendmail(myOldContext.studentEmail, recEmail, msg)
            myOldContext.myEmailIntent = 0
            myOldContext.contextOld = ""
            myOldContext.myFirstContext = ""
            myOldContext.myIntent = ""
            myOldContext.myOldTag = myOldContext.chatRestart
            return "email sent! you will hear from the wellbeing team shortly" + myOldContext.chatRestart
        except smtplib.SMTPAuthenticationError:
            myOldContext.myEmailIntent = 1
            return "Incorrect email or password: please enter your username"

In [ ]:
def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    #If the user input is correctly classified find matching intent/tag
    if results:
        #Loop through possible matches
        print(results)
        while results:
            for i in intents['intents']:
                #Get tag from first result(if matching)
                if i['tag'] == results[0][0]:
                    
                    if 'email' in i:
                        myOldContext.myEmailIntent = 1
                        myOldContext.staffEmail = i['email']
                        myOldContext.symptomType = i['symptom_type']
                        myOldContext.symptomProblem = i['symptom_problem']

                    if 'context_set' in i and 'context_filter' not in i and myOldContext.myFirstContext == "":
                        myOldContext.myOldTag = random.choice(i['responses'])
                        myOldContext.myFirstContext = i['context_set']
                        myOldContext.myIntent = i['intent']

                    try:
                        if ('intent' in i and i['context_filter'] == myOldContext.contextOld) or (i['tag'] == myOldContext.myTag):
                            myOldContext.myOldTag = random.choice(i['responses'])
                            myOldContext.myIntent = i['intent']
                            print(myOldContext.myIntent)

                    except KeyError:
                        print("Filter Not In Requested Object")

                    
                    print(myOldContext.myOldTag)
                    print(myOldContext.contextOld)

                    if 'context_filter' in i and i['context_filter'] == myOldContext.contextOld:
                        context[userID] = filtering(i, show_details)
                        if 'context_set' not in i and myOldContext.myEmailIntent < 1:
                            myOldContext.contextOld = ""
                            myOldContext.myIntent = ""
                            myOldContext.myFirstContext = ""
                            myOldContext.myOldTag = myOldContext.chatRestart
                            myResp = advancedResponse(i) + myOldContext.chatRestart
                        else:
                            myResp = advancedResponse(i)
                        print("reached")
                        return myResp
                    elif 'context_filter' not in i and myOldContext.contextOld != "" or (
                            'context_filter' in i and i['context_filter'] != myOldContext.contextOld):
                        return "I'm sorry? Can you repeat that?: " + myOldContext.myOldTag
                    else:
                        context[userID] = filtering(i, show_details)

                        if 'context_filter' not in i or \
                                (userID in context and 'context_filter' in i and i['context_filter'] == context[
                                    userID]):
                            if show_details:
                                return 'tag:', i['tag']
                            # a random response from the intent
                            return random.choice(i['responses'])

            results.pop(0)
    else:
        if myOldContext.myIntent != "":
            return "I'm sorry? Can you repeat that?: " + myOldContext.myOldTag
        else:
            return "I'm sorry? Can you repeat that?: " + myOldContext.myOldTag

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

In [ ]:
@app.route('/get')
def userChat():
    myStr = ""
    if ContextHistory.time > 600.00 and ContextHistory.myFirstInput != 2:
        ContextHistory.myFirstInput = 2

        myStr += qaCode.decode()

    userInput = request.args.get('msg')
    myTime()

    if myOldContext.myEmailIntent != 0:
        myStr = email(userInput)
    else:
        if myOldContext.myIntent != "":
            userInput = myOldContext.myIntent + userInput
        print(userInput)
        myStr += response(userInput)

        if ContextHistory.myFirstInput < 1:
            ContextHistory.myFirstInput = 1
    return str(myStr)

In [ ]:
@app.route('/')
def index():
    return render_template('index.html')

In [ ]:
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://46c8b77ffe89.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [15/Apr/2021 09:34:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2021 09:34:20] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2021 09:34:21] "GET /static/pop.mp3 HTTP/1.1" 206 -
127.0.0.1 - - [15/Apr/2021 09:34:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2021 09:34:33] "GET /get?msg=hello HTTP/1.1" 200 -


6.016000043018721e-06
hello
[('greeting', 0.99791116)]




127.0.0.1 - - [15/Apr/2021 09:34:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2021 09:34:43] "GET /get?msg=check%20my%20symptoms%20please HTTP/1.1" 200 -


9.212738607999995
check my symptoms please
[('symptoms', 0.9999342)]
Filter Not In Requested Object
Please specify what physical and/or mental anxiety symptoms are more prevalent whilst at university during covid-19! I.E. 'I feel inferior to other students' or 'I have feelings of nausea'



127.0.0.1 - - [15/Apr/2021 09:34:55] "GET /get?msg=I%20am%20starting%20to%20feel%20very%20sick HTTP/1.1" 200 -


21.62060266200001
userChoiceOneGrp_I am starting to feel very sick
[('physicalSymptom_MediumRisk', 0.9997011)]
physSymp_medRsk_
It is heavily advised that you meet with a member of the wellbeing team as a means to discuss your anxieties and symptom triggers in a professional and understanding environment as the physical symptoms you have specified that your state of anxiety has indicated that you may be suffering from a higher anxiety disorder in which is beyond the capabilities of the chatbot and indicates you may not be functioning/managing your stress/anxieties without the utilisation of professional help. Do you want me to redirect you to the wellbeing teams appointment page as to allow you to book an appointment with a member of the wellbeing team?
symptomMentalOrPhysical
reached
